# Stimmungsanalyse mit dem DB-Social-Media Datensatz

Der Datensatz wurde zur Stimmungsanalyse und Erkennung von relevanten Feedback erstellt.
Weitere Dokumentation und Downloads sind unter https://sites.google.com/view/germeval2017-absa/data?authuser=0 zu finden.


Der Datensatz wird mittels Logistischer Regression, FastText und Bert bearbeitet.
# Jupyter-Notebook

Jupyter-Notebooks sind interaktive Python-Scripte, in denen Markdown und sogar Latex zur Dokumentation verwendet werden kann. In diesem Abschnitt sollen einerseits allgemeine Informationen wie Tastenkombinationen und übliche Workflows, andererseits aber auch für diesen Workshop und die verwendete Hardware spezifische Informationen im Umgang mit Python und Jupyter-Notebooks geliefert werden.

### Wichtiges auf einen Blick (a.k.a. TL;DR)

- **beim Wechseln auf ein anderes Notebook immer den Kernel beenden (Kernel -> Shutdown)**
- **Shift + Enter** zum Ausführen der aktiven Zelle
- **Strg + Shift + p** zum Öffnen der Kommandopalette
- **Shift + o** zum Togglen des Zell-Scrollings
- bei Fehlermeldungen im Zweifel **Kernel neustarten (Kernel -> Restart)**
- "Hilfe, ich sehe den Markdown-Code" -> **Shift + Enter** in der entsprechenden Zelle
- "Hilfe, ich bekommen ResourceExhaustion / OutOfMemory (OOM) Fehler" -> **Kernel neustarten, Kernel von anderen noch laufenden Notebooks herunterfahren** (oben links auf das Jupyter-Logo klicken, dann auf den Tab "Running")
- "Hilfe, mein Notebook ist kaputt" -> siehe **Notebook "Wiederherstellung"**
- "Passiert da noch was?" -> ist der **Kreis oben rechts neben "Python 3", ausgefüllt und dunkel** dann ist der Kernel noch am Arbeiten, ist er nicht gefüllt dann ist der Kernel untätig. Die aktuell laufende Zelle ist die von oben gesehen erste bei der auf der linken Seite "In[*]" anstelle von z.B. "In[5]" steht. Es kann aber passieren dass sich der Kernel aufhängt, in dem Fall einfach oben auf __Kernel -> Interrupt__ und die Zelle erneut ausführen


### Überblick & Workflow

Die einzigen beiden Shortcuts die man sich eigentlich nur merken muss sind

- **Shift + Enter** zum Ausführen einer Zelle, und
- **Strg + Shift + p** zum Öffnen der Kommandopalette, von der aus man dann direkt Zugriff auf alle möglichen Befehle hat, inklusive entsprechender Shortcuts

Ein weiterer nützlicher Shortcut ist **Shift + o**, welcher das **Scrolling für Zellenoutput** umschaltet.

Zum **Editieren einer Zelle** genügt ein Doppelklick in die Zelle, bei Code-Zellen reicht es zum Beenden des Editiermodus einfach außerhalb der Zelle zu klicken, bei Dokumentationszellen (wie dieser hier) ist eine Ausführung der Zelle nötig um die Code-Ansicht zu verlassen.

Während eine Zelle ausgeführt wird, wechselt der Kernel-Indikator oben rechts neben "Python 3" von einem hellen Kreis mit dunklem Rand zu einem ausgefüllten dunklen Kreis und springt wieder zurück sobald die Ausführung beendet ist. Wenn mehrere Zellen gleichzeitig ausgeführt wurden, kann man an dem Label in der linken Spalte ablesen, ob die Zelle fertig ausgeführt wurde (**In [ZAHL]:**) oder ob sie gerade ausgeführt wird bzw. auf Ausführung wartet (**In [*]:**). Zusätzlich wird nach Ausführung einer Zelle die Zellenausgabe unterhalb der Zelle angezeigt.

Um den Überblick zu behalten kann es manchmal sinnvoll sein, die **Zellenausgabe zu löschen**. Dies kann u.a. auf diesen beiden Wegen erfolgen:

- oben auf Cell -> Current Outputs / All outputs -> Clear
- oben auf Kernel -> Restart & Clear Output

Der Kernel ist für die Ausführung des Python-Codes zuständig und behält den Kontext (also belegte Variablen, definierte Funktionen und belegter Speicher) seit dem letzten Kernel-(Neu)start. Dies kann zu Problemen führen wenn Zellen in anderer Reihenfolge ausgeführt werden oder Zellen übersprungen werden in denen Variablen oder Funktionen definiert werden die im weiteren Verlauf des Scripts benötigt werden, aber auch wenn **ein anderes Jupyter-Notebook gestartet wird**, da dafür ein weiterer Kernel gestartet wird.

Deswegen beim **Wechseln auf ein anderes Notebook** immer den **Kernel herunterfahren oder neustarten** (oben Kernel -> Restart/Shutdown).

In [1]:
import os, time, datetime, random

import pandas as pd
import numpy as np

import fasttext
import gensim

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.cuda.amp import GradScaler, autocast

import transformers
from transformers import BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import data_cleaner # pyscript data_cleaner.py

In [40]:
ROOT_DIR = os.path.abspath("./")
DATA_DIR = os.path.abspath(f"./dataset/")

TRAIN_DF= f"{DATA_DIR}/db_twitter/train_v1.4.tsv"
TEST_DF= f"{DATA_DIR}/db_twitter/dev_v1.4.tsv"
CLEAN_TRAIN_DF= f"{DATA_DIR}/db_twitter/train_clean.tsv"
CLEAN_TEST_DF= f"{DATA_DIR}/db_twitter/test_clean.tsv"

# 1. Datenanalyse
Zur Analyse laden wir den Testdatensatz ein und schauen uns die ersten fünf Zeilen an.

In [3]:
analyse_train = pd.read_csv(TRAIN_DF, delimiter='\t', header=None)
analyse_test = pd.read_csv(TEST_DF, delimiter='\t', header=None)

with pd.option_context('display.max_colwidth', 19):
    print("Die ersten 5 Zeilen in der Trainingsmenge:")
    print(analyse_train[:5],"\n")
    print("Die ersten 5 Zeilen in der Testmenge:")
    print(analyse_test[:5], "\n")
    
print(f"Nachricht der ersten Zeile der Trainingsmenge:\n{analyse_train[1].iloc[0]}\n")
print(f"Nachricht der ersten Zeile der Testmenge:\n{analyse_test[1].iloc[0]}")

Die ersten 5 Zeilen in der Trainingsmenge:
                    0                   1      2         3                   4
0  http://twitter....  @DB_Bahn ja, we...   True   neutral  Allgemein#Haupt...
1  http://twitter....  @nordschaf theo...   True  positive  Zugfahrt#Streck...
2  http://twitter....  Bahn verspätet ...   True  negative  Zugfahrt#Pünktl...
3  http://wirtscha...  Ihre Anfragen b...  False   neutral                 NaN
4  http://communit...  Kann ich mit de...   True   neutral  Allgemein#Haupt... 

Die ersten 5 Zeilen in der Testmenge:
                    0                   1      2         3                   4
0  http://www.face...  Re: Das Erste "...   True  negative  Allgemein#Haupt...
1  http://www.luft...  Effektiver Part...  False   neutral                 NaN
2  http://twitter....  #TelMi #telmi L...   True  negative  Sicherheit#Haup...
3  http://twitter....  @KuttnerSarah @...   True  negative  Gepäck#Haupt:ne...
4  http://twitter....  Probleme bei de...   True

## 1.1 Klassenverteilung
Die Verteilung der Klassen sieht wie folgt aus:

In [4]:
unique_train, counts_train = np.unique(analyse_train[3], return_counts=True)
unique_test, counts_test = np.unique(analyse_test[3], return_counts=True)

print(f"Trainingsmenge:\n{np.asarray((unique_train, counts_train)).T}")
print(f"Testmenge:\n{np.asarray((unique_test, counts_test)).T}")

Trainingsmenge:
[['negative' 5228]
 ['neutral' 14497]
 ['positive' 1216]]
Testmenge:
[['negative' 617]
 ['neutral' 1812]
 ['positive' 155]]


# 2. Daten bereinigen

Für unsere Aufgabe benötigen wir nur den Text der Posts und die Labels, daher nutzen wir nur die Spalten 1 und 3.

Um Erwähnungen zu vereinfachen, wird jeder Twitterusername durch den Text 'twitterusername' ersetzt. @DB_Bahn wird besonders gekennzeichnet, da er hier von besonderer Bedeutung ist.

Zudem werden Satzzeichen entfernt und Emojis durch festgelegte Wörter ersetzt.

In [45]:
train_data = pd.read_csv(TRAIN_DF, delimiter='\t', usecols=[1,3], header=None).rename(columns={1:0,3:1})
test_data = pd.read_csv(TEST_DF, delimiter='\t', usecols=[1,3], header=None).rename(columns={1:0,3:1})

print(train_data[:5])

                                                   0         1
0  @DB_Bahn ja, weil in Wuppertal Bauarbeiten sin...   neutral
1  @nordschaf theoretisch kannste dir überall im ...  positive
2  Bahn verspätet sich..gleich kommt noch jemand ...  negative
3  Ihre Anfragen brachten uns zu neuen Leistungen...   neutral
4  Kann ich mit dem DB Geschenk Ticket den ICE Sp...   neutral


In [46]:
print("Datensätze werden bereinigt, dies kann einige Zeit in Anspruch nehmen...")
clean_train_data = train_data.apply(data_cleaner.clean_text)
clean_test_data = test_data.apply(data_cleaner.clean_text)
print("Datensätze bereinigt.")

print(clean_train_data[:5])

Datensätze werden bereinigt, dies kann einige Zeit in Anspruch nehmen...
Datensätze bereinigt.
                                                   0         1
0  dbusername ja weil in wuppertal bauarbeiten si...   neutral
1  twitterusername theoretisch kannste dir uebera...  positive
2  bahn verspaetet sich annoyeddots gleich kommt ...  negative
3  ihre anfragen brachten uns zu neuen leistungen...   neutral
4  kann ich mit dem db geschenk ticket den ice sp...   neutral


Die bereinigten Datensätze sind bereits gespeichert und können im Folgenden geladen werden.
Ein Vergleich der originalen und bereinigten Daten kann hier angesehen werden. Über das Argument in `.iloc[]`kann der Eintrag gewählt werden. 

In [48]:
print(f"original: {train_data[0].iloc[2]}")
print(f"bereinigt: {clean_train_data[0].iloc[2]}")

original: Bahn verspätet sich..gleich kommt noch jemand und drückt mir ein riesiges Kreuz in die Hand, das ich rumschleppen muss
bereinigt: bahn verspaetet sich annoyeddots gleich kommt noch jemand und drueckt mir ein riesiges kreuz in die hand das ich rumschleppen muss


## Bereinigte Trainings und Testdaten laden
**Wird verwendet, falls das Bereinigen übersprungen werden soll**

In [49]:
clean_train_data = pd.read_csv(CLEAN_TRAIN_DF, delimiter='\t', header=None)
clean_test_data = pd.read_csv(CLEAN_TEST_DF, delimiter='\t', header=None)

# 3. Logistische-Regression mit Gensim

Der Trainings und Testdatensatz wird mit Gensim zum vectorisieren vorverarbeitet:

In [50]:
def read_corpus(fname):
    for i in range(0,fname.shape[0]):
        tokens = gensim.utils.simple_preprocess(str(fname[0].iloc[i]))
        yield gensim.models.doc2vec.TaggedDocument(tokens, str(fname[1].iloc[i]))
                
train_corpus = list(read_corpus(clean_train_data))
test_corpus = list(read_corpus(clean_test_data))       

print("Trainingsdaten:")
print(train_corpus[:1])
print("\nTestdaten:")
print(test_corpus[:1])

Trainingsdaten:
[TaggedDocument(words=['dbusername', 'ja', 'weil', 'in', 'wuppertal', 'bauarbeiten', 'sind', 'soweit', 'bin', 'ich', 'auch', 'aber', 'wieso', 'nur', 'am', 'wochenende', 'und', 'grade', 'jetzt'], tags='neutral')]

Testdaten:
[TaggedDocument(words=['re', 'emote', 'das', 'erste', 'ich', 'fahre', 'nicht', 'mit', 'der', 'bahn', 'nach', 'erzaehlungen', 'von', 'freunden', 'scheint', 'es', 'ein', 'zentrales', 'problem', 'zu', 'sein', 'es', 'kann', 'nicht', 'sei', 'dass', 'jemand', 'in', 'hannover', 'umsteigen', 'muss', 'um', 'in', 'einen', 'zug', 'auch', 'ohne', 'klimaanlage', 'nach', 'berlin', 'zu', 'fahren', 'es', 'war', 'aber', 'bei', 'dem', 'letzten', 'hitzehoch', 'das', 'liegt', 'am', 'hersteller'], tags='negative')]


## 3.1 Doc2Vec-Modell erzeugen
Gensim wird für ganze Sätze konfiguriert(docs) und das Wörterbuch wird erstellt. "min_count" ignoriert alle Wörter, die seltener als definiert vorkommen.

In [51]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=15)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Zunächst werden Text und Labels für einfache Weiterverwendung extrahiert. Danach werden die Trainingssätze vom Doc2Vec-Modell in Vektoren umgewandelt und den Traininglabels eindeutige Zahlen zugeordnet. 

In [52]:
train_regressors, train_targets = zip(*[(doc.words, doc.tags) for doc in train_corpus])

# converting training sentences into vectors
train_x = np.asarray([model.infer_vector(regressor) for regressor in train_regressors])

# converting training labels to unique numbers 
target_name_idx = {target:i for i, target in enumerate(np.unique(train_targets))}
train_y = np.vectorize(target_name_idx.get)(train_targets)
print(f"{target_name_idx}")

{'negative': 0, 'neutral': 1, 'positive': 2}


## 3.2 Logistische Regression
Die erwarteten Labels werden an die Logistische regression gegeben

In [53]:
logreg = LogisticRegression(max_iter=2000)
logreg.fit(train_x, train_y)

LogisticRegression(max_iter=2000)

## 3.3 Genauigkeit der Testmenge
Hier wird die Testmenge von dem Modell analysiert. Die Zusammenfassung wird von SKLearn angezeigt:

In [54]:
test_regressors, test_targets = zip(*[(doc.words, doc.tags) for doc in test_corpus])

# converting test sentences into vectors
test_x = np.asarray([model.infer_vector(regressor) for regressor in test_regressors])
# converting test labels to unique numbers 
test_y = np.vectorize(target_name_idx.get)(test_targets)

# predict labels of test sentences
preds = logreg.predict(test_x)

print(classification_report(test_y, preds, target_names=target_name_idx.keys()))

              precision    recall  f1-score   support

    negative       0.50      0.32      0.39       617
     neutral       0.75      0.87      0.81      1812
    positive       0.23      0.10      0.14       155

    accuracy                           0.70      2584
   macro avg       0.49      0.43      0.45      2584
weighted avg       0.66      0.70      0.67      2584



# 4 FastText

[FastText](https://fasttext.cc/) ist ein Framework von Facebook zum Klassifizieren von Texten. Es ist effizient und kann sogar auf Mobilgeräten verwendet werden.


## 4.1 Vorverarbeitung für FastText
Zuerst werden die Kommentare mit Labels versehen. Sie werden aus dem Datensatz in diese Form gebracht: <br>
"\__label\__<positive/neutral/negative> >Kommentar<".
    
Die ersten zwei Einträge werden angezeigt.

In [55]:
labeledList_train = [f"__label__{row[1]} {row[0]}" for _, row in clean_train_data.iterrows()]
labeledList_test = [f"__label__{row[1]} {row[0]}" for _, row in clean_test_data.iterrows()]
print(labeledList_train[:2])

with open(f"{ROOT_DIR}/fasttext_train.txt", 'w') as f:
      f.write("\n".join(labeledList_train))

with open(f"{ROOT_DIR}/fasttext_test.txt", 'w') as f:
      f.write("\n".join(labeledList_test))

['__label__neutral dbusername ja weil in wuppertal bauarbeiten sind soweit bin ich auch aber wieso nur am wochenende und grade jetzt', '__label__positive twitterusername theoretisch kannste dir ueberall im koelner stadtbereich was suchen mit der kvb sbahn kommt man ueberall fix hin']


## 4.2 Training
Anschließend wird mit FastText der Text trainiert. Die Anzahl der Epochen, Lernrate ([0.1,1]) und NGram-länge ([1,5]) kann angepasst werden.

In [56]:
model = fasttext.train_supervised(input=f"{ROOT_DIR}/fasttext_train.txt", epoch=20, lr=0.2, wordNgrams=4)

Read 1M words
Number of words:  96976
Number of labels: 3
Progress: 100.0% words/sec/thread:  119111 lr:  0.000000 avg.loss:  0.183315 ETA:   0h 0m 0s


## 4.3 Test des Modells
Um das Modell zu testen, wird der Testdatensatz übergeben.

Die Ausgabe ist wie folgt Formatiert: (Samples, Precision, Recall)

In [57]:
model.test(f"{ROOT_DIR}/fasttext_test.txt")

(2584, 0.7801857585139319, 0.7801857585139319)

## 4.4 Einzelinferenz
Hier können eigene Sätze mit dem Modell getestet werden

In [58]:
to_predict="@DB_Bahn Heute mal wieder viel zu spät :("

#preprocess input
df = pd.DataFrame([to_predict])
df = df.apply(data_cleaner.clean_text)
to_predict = df[0].item()
print(f"'{to_predict}'")

model.predict(to_predict, k=-1, threshold=0.3)

'dbusername heute mal wieder viel zu spaet sadsmiley'


(('__label__negative',), array([1.00000966]))


# 5 Bert
[Bert](https://huggingface.co/transformers/model_doc/bert.html) (Bidirectional Encoder Representations from Transformers) ist ein Modell, das nicht nur die Wörter in einem Text, sondern auch den Kontext analysiert.

In diesem Notebook wird Pytorch als Backend verwendet.

In [59]:
clean_train_data = pd.read_csv(CLEAN_TRAIN_DF, delimiter='\t', header=None)
clean_test_data = pd.read_csv(CLEAN_TEST_DF, delimiter='\t', header=None)
print(clean_train_data[:2])

                                                   0         1
0  dbusername ja weil in wuppertal bauarbeiten si...   neutral
1  twitterusername theoretisch kannste dir uebera...  positive


## 5.1 Preprocessing
Das Modell erwartet die Labels als Ordinale, daher werden sie umgewandelt.

In [60]:
# convert labels to ordinals
labels_to_ordinal = {target:i for i, target in enumerate(np.unique(clean_train_data[1]))}
print(f"ordinale labels: {labels_to_ordinal}")
        
bert_train = pd.DataFrame({'label': np.vectorize(labels_to_ordinal.get)(clean_train_data[1]),
                         'text':clean_train_data[0]})

bert_test = pd.DataFrame({'label': np.vectorize(labels_to_ordinal.get)(clean_test_data[1]),
                         'text':clean_test_data[0]})

ordinale labels: {'negative': 0, 'neutral': 1, 'positive': 2}


Der Text wird mit Hilfe von BERT in Tokens mit Start- und Stopp-Symbolen umgewandelt. Es werden nur Eingaben von 512 Zeichen unterstützt.

In [61]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased", do_lower_case=True)
print("Token werden erstellt, dies kann einige Zeit in anspruch nehmen..")
train_tokenized = bert_train['text'].astype(str).apply((lambda x: tokenizer.encode(x[:512], add_special_tokens=True)))
test_tokenized = bert_test['text'].astype(str).apply((lambda x: tokenizer.encode(x[:512], add_special_tokens=True)))
print("Token erstellt!")

print(f"\nText: {clean_train_data[0].iloc[0]}")
print(f"\nAls Token: {train_tokenized[0]}")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

Token werden erstellt, dies kann einige Zeit in anspruch nehmen..
Token erstellt!

Text: dbusername ja weil in wuppertal bauarbeiten sind soweit bin ich auch aber wieso nur am wochenende und grade jetzt

Als Token: [3, 9, 5655, 212, 1431, 3278, 982, 50, 25, 6221, 14779, 3703, 3085, 287, 4133, 4058, 1169, 194, 386, 6724, 26910, 356, 235, 743, 280, 765, 42, 5086, 57, 1868, 4]


## 5.2 Padding der Daten
Da auf die Daten möglichst performant zugegriffen werden soll, werden sie als Arrays gespeichert. Die längste Zeichenkette gibt daher die größe der Arrays vor. Alle anderen werden am ende mit 0 aufgefült. Die `Attention Mask` filtert die aufgefüllten Werte später raus.

In [62]:
max_len = 0
for i in train_tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
for i in test_tokenized.values:
    if len(i) > max_len:
        max_len = len(i)        


train_padded = np.array([i + [0]*(max_len-len(i)) for i in train_tokenized.values])
train_attention_mask = np.where(train_padded != 0, 1, 0)

test_padded = np.array([i + [0]*(max_len-len(i)) for i in test_tokenized.values])
test_attention_mask = np.where(test_padded != 0, 1, 0)

## 5.3 Daten für GPU vorbereiten
Hier werden Datensätze zum Training und Testen erzeugt. Übergeben werden die Token-Sätze, Masken und Label. Die `Dataloader` laden die Datensätze mit der Größe `BATCH_SIZE` wenn benötigt auf die GPU, um den GPU-Speicher nicht durch den Datensatz auszulasten. In diesem Fall werden gleichzeitig 50 Einträge verarbeitet.

In [63]:
train_dataset = TensorDataset(torch.tensor(train_padded),
                             torch.tensor(train_attention_mask),
                             torch.tensor(list(bert_train['label'])))

test_dataset = TensorDataset(torch.tensor(test_padded),
                            torch.tensor(test_attention_mask),
                            torch.tensor(list(bert_test['label'])))

BATCH_SIZE = 32
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), 
            batch_size = BATCH_SIZE
        )

validation_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = BATCH_SIZE
        )

## 5.4 Modell erstellen
Es wird ein BERT Modell mit 12 Layern erzeugt, die Anzahl an Labels muss mit dem Datensatz übereinstimmen (hier "positive", "neutral", "negative").

Der "cuda()" Aufruf verschiebt das Modell auf die GPU

In [69]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-german-cased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## 5.5 Optimizer und Lernrate
Als Optimizer verwenden wir Adam, da dieser sehr schnell konvergiert.
Um möglichst viele Daten gleichzeitig zu verarbeiten, verwenden wir `mixed-precision`, d.h. es werden wenn möglich Float16 Werte anstelle von Float32 Werten verwendet, um Ressourcen einzusparen. Details zu `mixed-precision` mit PyTorch: https://pytorch.org/docs/stable/notes/amp_examples.html 

In [70]:
optimizer = Adam(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5,
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
scaler = GradScaler() # necessary for mixed precision training

## 5.5 Training vorbereiten
Die Anzahl an Epochen kann über `NUM_EPOCHS` eingestellt werden. Die Lernrate wird über einen scheduler gesteuert.
Die definierten Funktionen dienen zur Darstellung des Fortschritts im Training.

In [71]:
NUM_EPOCHS = 5

total_steps = len(train_dataloader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

## 5.6 Modell Training
Der Trainingsprozess bestet aus der Trainingsphase mit der Trainingsmenge und Validierungsphase mit der Validierungsmenge.

In [72]:
# use cuda device
device = torch.device("cuda")

# same seed for reproducibility
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# training statistics 
training_stats = []
# time measure
total_t0 = time.time()

for epoch in range(NUM_EPOCHS):
    # ========================================
    # Training phase
    # ========================================
    print(f'\n======== Epoch {epoch+1}/{NUM_EPOCHS} ========\nTraining...')

    #training time for epoch
    t0 = time.time()

    # reset total loss
    total_train_loss = 0

    # change model mode to training
    model.train()

    for step, (tokens, att_mask, labels) in enumerate(train_dataloader):
        # Progress update every 20 batches.
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0) # elapsed time
            print(f'  Batch {step:>5,}  of  {len(train_dataloader):>5,}.    Elapsed: {elapsed}.')

        tokens = tokens.to(device)
        att_mask = att_mask.to(device)
        labels = labels.to(device)

        # clear calculated gradiends
        optimizer.zero_grad()    
        
        # calculate gradients
        with autocast():
            # train model with current batch
            outputs = model(tokens, token_type_ids=None, attention_mask=att_mask, labels=labels)
            loss, logits = outputs.loss, outputs.logits
            
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient
        scaler.step(optimizer)
        scaler.update()
        
        # training loss of all batches for statistics
        total_train_loss += loss.item()
        
        # Update the learning rate
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print(f"  Average training loss: {avg_train_loss:.2f}\n  Training epoch took: {training_time}")
        
        
    # ========================================
    #  Validation phase
    # ========================================
    print("\n Running Validation...")

    t0 = time.time()
    # put the model in evaluation mode
    model.eval()

    # tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for (tokens, att_mask, labels) in validation_dataloader:
        tokens = tokens.to(device)
        att_mask = att_mask.to(device)
        labels = labels.to(device)
        
        # constructing the compute graph is only needed for training
        with torch.no_grad():        
            # the documentation for this model function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(tokens, token_type_ids=None, attention_mask=att_mask, labels=labels)
            loss, logits = outputs.loss, outputs.logits
            
        # accumulate the validation loss.
        total_eval_loss += loss.item()

        # move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        labels = labels.to('cpu').numpy()

        # calculate the accuracy 
        total_eval_accuracy += flat_accuracy(logits, labels)
        
    # report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    # calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print(f" Validation Accuracy: {avg_val_accuracy:.2f}")
    print(f"  Validation Loss: {avg_val_loss:.2f}")
    print(f"  Validation Time: {validation_time}")

    # record all statistics from this epoch.
    training_stats.append({
            'epoch': epoch + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("\nTraining complete!")
print(f"Total training took {format_time(time.time()-total_t0)} (h:mm:ss)")


======== Epoch 1/5 ========
Training...
  Batch    40  of    655.    Elapsed: 0:00:07.
  Batch    80  of    655.    Elapsed: 0:00:14.
  Batch   120  of    655.    Elapsed: 0:00:21.
  Batch   160  of    655.    Elapsed: 0:00:28.
  Batch   200  of    655.    Elapsed: 0:00:34.
  Batch   240  of    655.    Elapsed: 0:00:41.
  Batch   280  of    655.    Elapsed: 0:00:48.
  Batch   320  of    655.    Elapsed: 0:00:54.
  Batch   360  of    655.    Elapsed: 0:01:01.
  Batch   400  of    655.    Elapsed: 0:01:08.
  Batch   440  of    655.    Elapsed: 0:01:14.
  Batch   480  of    655.    Elapsed: 0:01:21.
  Batch   520  of    655.    Elapsed: 0:01:27.
  Batch   560  of    655.    Elapsed: 0:01:34.
  Batch   600  of    655.    Elapsed: 0:01:41.
  Batch   640  of    655.    Elapsed: 0:01:47.
  Average training loss: 0.59
  Training epoch took: 0:01:50

 Running Validation...
 Validation Accuracy: 0.80
  Validation Loss: 0.50
  Validation Time: 0:00:10

======== Epoch 2/5 ========
Training...
  B

## 5.7 Einzelinferenz
Hier können eigene Sätze mit dem Modell getestet werden

In [74]:
to_predict="@DB_Bahn heute wieder später"
ordinal_to_labels = {v: k for k, v in labels_to_ordinal.items()}

#preprocess input
df=pd.DataFrame([to_predict])
df=df.apply(data_cleaner.clean_text)
to_predict=(df[0].item())

print(f"Vorverarbeiteter Text: '{to_predict}'")

tokenized=tokenizer.encode(to_predict[:512])
print(f"Tokenized: {tokenized}")

#predict input
tokens = torch.tensor([tokenized]).cuda()
with torch.no_grad(): 
    model.eval()
    output = model(tokens, token_type_ids=None)
    logits = output.logits

prediction = np.argmax(logits.detach().cpu().numpy(), axis=1)[0]
print(f"Ergebnis: {ordinal_to_labels[prediction]}")

Vorverarbeiteter Text: 'dbusername heute wieder spaeter'
Tokenized: [3, 9, 5655, 212, 1431, 1138, 525, 338, 4173, 60, 4]
Ergebnis: negative


In [75]:
 torch.cuda.empty_cache()